In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_df  = pd.read_csv('../input/tabular-playground-series-jun-2021/train.csv')
test_df = pd.read_csv('../input/tabular-playground-series-jun-2021/test.csv')


In [ ]:
train_df.head()

In [ ]:
test_df.head()

In [ ]:
train_df.isna().sum(),test_df.isna().sum()

In [ ]:
X = train_df.drop(columns = ['target'])
y = train_df['target']

> **Check for collinearity **

In [ ]:
from sklearn.preprocessing import StandardScaler

scalar = StandardScaler()
X_scaled = scalar.fit_transform(X)


from statsmodels.stats.outliers_influence import variance_inflation_factor
vif = pd.DataFrame()
vif["vif"] = [variance_inflation_factor(X_scaled,i) for i in range(X_scaled.shape[1])]
vif["Features"] = X.columns

In [ ]:
 vif.loc[vif['vif']>5.0].count()

No collinearity

 Vizualization for any outliers

In [ ]:
 import matplotlib.pyplot as plt
import seaborn as sns
fig,ax = plt.subplots(figsize=(50,10))

sns.boxplot(data = X, width = 1, ax = ax, fliersize = 10)
plt.xticks(rotation=90)
plt.ylabel('values') 

In [ ]:
#No outliers are present. So Lets build the model

In [ ]:
import tensorflow
from tensorflow.keras import Sequential
from tensorflow.keras.layers import InputLayer
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Dense
from tensorflow.keras.metrics import AUC
from keras.layers import Dropout


In [ ]:
 
x_test = test_df.iloc[:,:]


In [ ]:
train_df['target'].unique()

In [ ]:
df = train_df.copy()

Deal with categorical values presnet in the label column. apply one hot encoding

In [ ]:
from sklearn.preprocessing import OneHotEncoder
# creating one hot encoder object 
onehotencoder = OneHotEncoder()
#reshape the 1-D country array to 2-D as fit_transform expects 2-D and finally fit the object 
X_new = onehotencoder.fit_transform(df.target.values.reshape(-1,1)).toarray()
#To add this back into the original dataframe 
dfOneHot = pd.DataFrame(X_new) 
df_new = pd.concat([df, dfOneHot], axis=1)

In [ ]:
df_new.head()

In [ ]:
df_new = df_new.drop(['target'],axis=1)

In [ ]:
X2 = df_new.iloc[:,:76]
y2 = df_new.iloc[:,76:]

In [ ]:
 y2.columns

In [ ]:

from sklearn.model_selection import train_test_split

x_train, x_valid, y_train, y_valid = train_test_split(X2, y2, train_size=0.8, test_size=0.2, random_state=48)

In [ ]:
def custom_model2():
    model = Sequential(InputLayer([76]))
    model.add(Dropout(0.2 ))
    model.add(BatchNormalization())
    model.add(Dense(8, kernel_initializer='he_normal',activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(16,kernel_initializer='he_normal' ,activation='relu'))
    model.add(Dropout(0.2 ))
    model.add(Dense(9, activation='softmax'))
                        
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy',AUC()])
    return model

model_eval = custom_model2()
history = model_eval.fit(
     x_train, y_train,
    validation_data=(x_valid, y_valid),
    batch_size=100,
    epochs=100, verbose=2
    )

In [ ]:
model_eval = custom_model2()
history = model_eval.fit(
     X2,y2,
    batch_size=50,
    epochs=100, verbose=2
    )

In [ ]:
predicted = model_eval.predict(x_test)
output = pd.DataFrame(predicted)
output = output.rename(columns={i:f'Class_{i+1}' for i in range(9)})
output = output.rename_axis("id", axis='rows')
idcol = pd.read_csv('../input/tabular-playground-series-jun-2021/test.csv')
idcol = idcol.iloc[:,0]
output = pd.concat([idcol, output], axis=1)
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")